In [ ]:
# XGBOOST WITH SENTIMENT 18
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
import matplotlib.pyplot as plt


----------------------------
2. Feature Engineering: Creating Lag Features for BTC Data Only
----------------------------
Create lag features for BTC data (close and volume)
merged_df['close_lag1'] = merged_df['close'].shift(1)
merged_df['volume_lag1'] = merged_df['volume'].shift(1)

# Use sentiment as-is (current day's sentiment, not lagged)
merged_df['sentiment_no_lag'] = merged_df['avg_sentiment']


# Define features and target
features = ['close_lag1', 'volume_lag1', 'sentiment_no_lag']
target = 'close'

# ----------------------------
# 3. Train-Test Split
# ----------------------------
# Split data based on datetime
train_data = merged_df[(merged_df['datetime'] >= '2018-01-01') & (merged_df['datetime'] < '2019-02-01')].copy()
test_data = merged_df[merged_df['datetime'] >= '2019-02-01'].copy()

X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

# ----------------------------
# 4. Model Building and Training with XGBoost
# ----------------------------
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Generate predictions on the test set
predictions = xgb_model.predict(X_test)

# ----------------------------
# 5. Performance Evaluation
# ----------------------------
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mse)
mape = mean_absolute_percentage_error(y_test, predictions) * 100
r2 = r2_score(y_test, predictions)

print("\nPerformance Metrics for the XGBoost Model:")
print(f"MSE: {mse:.2f}")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")
print(f"R2 Score: {r2:.2f}")

# ----------------------------
# 6. Visualization: Predicted vs Actual Close Prices
# ----------------------------
plt.figure(figsize=(12, 6))
plt.plot(test_data['datetime'], y_test, label='Actual Close Price', color='blue')
plt.plot(test_data['datetime'], predictions, label='Predicted Close Price', color='red', linestyle='--')
plt.xlabel('Date')
plt.ylabel('BTC Close Price')
plt.title('BTC Close Price Prediction: Actual vs Predicted (XGBoost)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
